In [11]:
import numpy as np

In [17]:
def E_P(stock_price,security_price, interest, vol, K, T, simulation_num, barrier,return_rate):
    # 함수 하나로 price가 바로 나올 수 있게 simulation 수도 함수 안에 포함해서 simulation을 한 방에 실행
    dt = 1/252 # 이것을 무엇으로 설정할 지에 대한 생각이 필요할 듯
    payoff = [security_price*(1+return_rate*i/2) for i in range(7)] #각 조건에서 payoff들을 담은 list
    num=[0,0,0,0,0,0,0] # 각 조건에서의 횟수를 담을 list
    s0 = stock_price
    r = interest
    sigma = vol
    discount = [np.exp(-r*T*i/6) for i in range(1,7)]
    ep = np.random.normal(0,1)
    result = 0
    t = T*252 #중간평가가 반년마다 있기 때문에 2 곱해줌, Q. 모든 path를 고려해야할 필요있나 ? 중간 평가 당시의 기초자산의 가격만 고려하면 되겠지 ? 노 dt를 1/252로 잡았기 때문에 일자로 해야함
    # Hull 책 307쪽에 명시된 GBM을 사용함
    for i in range(simulation_num+1):
        s = s0 #초기화
        for k in range(t):
            s = s * np.exp((r-0.5*sigma**2)*dt + sigma*ep*np.sqrt(dt))

            # barrier 아래로 내려가면 바로 naked가 return 되도록 가장 상위에서 설정
            if s > s0 * barrier:
                if k == t/6: #1차 조기상환
                    if s >= s0 * K[1]:
                        num[0] += 1
                if k == t/3: #2차 조기상환
                    if s >= s0 * K[2]:
                        num[1] += 1
                    
                if k == t/2: #3차 조기상환
                    if s >= s0 * K[3]:
                        num[2] += 1
                
                if k == t*2/3: #4차 조기상환
                    if s >= s0 * K[4]:
                        num[3] += 1
                        
                if k == t*5/6: #5차 조기상환
                    if s >= s0 * K[5]:
                        num[4] += 1
                        
                if k == t: # 만기
                    if s >= s0 * K[6]:
                        num[5] += 1
            else:
                payoff.append(s)
                num[6] += 1
                break
    
    
    weight = [num[i]/simulation_num*discount[i] for i in range(0,6)]
    weight.append(num[6]/simulation_num*discount[5])
    result = weight[0]*payoff[0] + weight[1]*payoff[1] + weight[2]*payoff[2] + weight[3]*payoff[3] + weight[4]*payoff[4] + weight[5]*payoff[5] + weight[6]*payoff[6]
    
    return result


In [13]:
# 각 parameter들 설정
stock_price = 726000 # 23.5.8자 LG화학 종가, 계약서에서 종가를 사용한다고 명시하였기 때문에 종가를 사용, Q. 보통 종가를 사용하나 ?
security_price = 10000
interest = 0.03340 # 금투협 채권정보센터 나이스 피앤아이 5.8일자 1년물 채권의 수익률
vol = 0.4217
K = np.array([0.9,0.85,0.8,0.8,0.75,0.75]) # 각 조기행사 조건
T = 3 # 23~26년까지 3년
simulation_num = 1000 # 몇 번이 최적의 simul 횟수일까 ?
barrier = 0.4 # ELS의 가치가 없어지는 지점. 40%로 설정
return_rate = 0.1136 # 연 수익률 11.36%


In [19]:
price = E_P(stock_price, security_price, interest, vol, K, T, simulation_num, barrier, return_rate)
print(price)

12141.763565562598
